In [4]:
import pandas as pd
import numpy as np

In [5]:
cutoffs = pd.read_csv('clgBranchCutoff.csv')
cutoffs = cutoffs.iloc[::2]
cutoffs = cutoffs.drop(columns = ['Unnamed: 0', 'GNT2S', 'LNT2S', 'ORPHAN', 'EWS', 'TFWS', 'ABCD', 'EFGH', 'Unnamed: 16', 'Unnamed: 17'])
cutoffs = cutoffs.iloc[:681]

In [16]:
clgBranch = pd.read_csv('clgNameBranch.csv')
clg = clgBranch.iloc[::2]
branch = clgBranch.iloc[1::2]
clg = list(clg['clg'])
branch = list(branch['clg'])
cutoffs['ClgName'] = clg
cutoffs['Branch'] = branch
colnames = ['ClgName', 'Branch', 'GOPENS', 'GSCS', 'GSTS', 'GNT1S', 'GOBCS', 'LOPENS', 'LOBCS', 'PWDOPENS']
cutoffs = cutoffs.reindex(columns=colnames)
cutoffs
colname = ['GOPENS', 'GSCS', 'GSTS', 'GNT1S', 'GOBCS', 'LOPENS', 'LOBCS', 'PWDOPENS']
cutoffs = cutoffs.replace(r'\r.*', '', regex=True)
cutoffs = cutoffs.replace(r'.*- ', '', regex=True)
cutoffs

,ClgName,Branch,GOPENS,GSCS,GSTS,GNT1S,GOBCS,LOPENS,LOBCS,PWDOPENS
0,"Government College of Engineering, Amravati",Civil Engineering,15776,17165,49645,21718,16685,9318,16120,NaN
2,"Government College of Engineering, Amravati",Electronics and Telecommunication Engg,9087,13812,47845,10529,10401,10835,13745,NaN
4,"Sant Gadge Baba Amravati University,Amravati",Food Technology,61213,40902,79193,36588,NaN,NaN,NaN,NaN
6,"Sant Gadge Baba Amravati University,Amravati",Oil and Paints Technology,47089,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Sant Gadge Baba Amravati University,Amravati",Paper and Pulp Technology,72318,82778,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1352,Pimpri Chinchwad Education Trust's Pimpri Chin...,Mechanical Engineering,75713,87570,NaN,NaN,NaN,NaN,NaN,NaN
1354,Dr.D.Y.Patil College Of Engineering & Innovati...,Computer Engineering,87856,59128,NaN,NaN,NaN,NaN,NaN,NaN
1356,Dr. D Y Patil Pratishthan's College of Enginee...,Computer Science and Engineering,88471,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1358,Dr. D Y Patil Pratishthan's College of Enginee...,Electrical Engineering,I,84548,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
cutoffs.to_csv("cutoffs.csv")

In [126]:
def extract_score_rank(string):
    info = list(string.split())
    info[1] = info[1][1:-1]
    return info[0],info[1]

def extract_name_town_city(sample):
    res = sample.split(',')
    name, town, city = '', '', ''
    if len(res) >= 1: name = res[0]
    if len(res) >= 2: town = res[1]
    if len(res) >= 3: city = res[2]
    name
    return name, town, city

def extraction(file_names: list):
    finalTable = pd.DataFrame(columns=['Rank', 'Scores', 'Name', 'Branch', 'Town', 'City'])
    l = []
    for file_name in file_names:
        table = pd.read_csv(file_name)
        print(table.columns)
        for i in range(table.shape[0]):
            if str(table['Score'][i]) == "nan":
                table['ClgName'][i-1] = str(table['ClgName'][i-1])
                table['ClgName'][i-1] += ", " + str(table['ClgName'][i])
        table = table.replace(r'.*- ', '', regex=True)

        table = table.drop(columns=['ChoiceCode', 'A', 'B'])
        table = table.dropna(subset=['Score'])
        
        table['Rank'], table['Scores'] = zip(*table['Score'].map(extract_score_rank))
        table['Name'], table['Town'], table['City'] = zip(*table['ClgName'].map(extract_name_town_city))
        
        table = table.replace(r'.*- ', '', regex=True)
        
        table = table.drop(columns=['Score', 'ClgName'])
        table = table.reindex(columns=['Rank', 'Scores', 'Name', 'Branch', 'Town', 'City'])
        l.append(table)
    res = l[0].append(l[1:])
    return res

In [127]:
file_names = ['jeecap1.csv', 'jeecap2.csv', 'jeecap1_2021.csv', 'jeecap2_2021.csv']
df = extraction(file_names)

Index(['Score', 'ChoiceCode', 'ClgName', 'Branch', 'A', 'B'], dtype='object')
Index(['Score', 'ChoiceCode', 'ClgName', 'Branch', 'A', 'B'], dtype='object')


<ipython-input-126-8120cbf68654>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['ClgName'][i-1] = str(table['ClgName'][i-1])
<ipython-input-126-8120cbf68654>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['ClgName'][i-1] += ", " + str(table['ClgName'][i])
<ipython-input-126-8120cbf68654>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['ClgName'][i-1] = str(table['ClgName'][i-1])
<ipython-input-126-8120cbf68654

Index(['Score', 'ChoiceCode', 'ClgName', 'Branch', 'A', 'B'], dtype='object')
Index(['Score', 'ChoiceCode', 'ClgName', 'Branch', 'A', 'B'], dtype='object')


<ipython-input-126-8120cbf68654>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['ClgName'][i-1] = str(table['ClgName'][i-1])
<ipython-input-126-8120cbf68654>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['ClgName'][i-1] += ", " + str(table['ClgName'][i])
<ipython-input-126-8120cbf68654>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['ClgName'][i-1] = str(table['ClgName'][i-1])
<ipython-input-126-8120cbf68654

In [128]:
df.to_csv('finaljeecap.csv')
moreinfo = pd.read_csv('moreclgdata.csv')
moreinfo.drop(moreinfo.iloc[:, 15:],inplace=True, axis=1)
df = pd.merge(df, moreinfo, left_on='Name', right_on='Name')
df = df.drop(columns=['Courses', 'City_x'])

c:\users\mohil\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_n

In [131]:
df.to_csv('collegesMerged.csv')
df['Scores'] = df['Scores'].astype(float)
df = df.sort_values('Scores', ascending=False)
import random
for i in range(len(list(df.Rating))):
    if i < 300:
        df['Rating'][i] = random.uniform(3.5, 4)
    elif i >= 300 and i < 700:
        df['Rating'][i] = random.uniform(3.0, 3.5)
    elif i >= 700 and i < 850:
        df['Rating'][i] = random.uniform(2, 3)
    else:
        df['Rating'][i] = random.uniform(1, 2)
df['Rating'] = df['Rating'].astype(float)
df = df.round({'Rating':2})

<ipython-input-131-e10850a2961c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rating'][i] = random.uniform(3.5, 4)
<ipython-input-131-e10850a2961c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rating'][i] = random.uniform(3.0, 3.5)
<ipython-input-131-e10850a2961c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rating'][i] = random.uniform(2, 3)
<ipython-input-131-e10850a2961c>:13: SettingWithCopyWarning: 
A value

In [133]:
df.to_csv('collegesMerged.csv')
